In [1]:
import pandas as pd

#data collection
from bs4 import BeautifulSoup
import requests
import re

In [2]:
url = 'https://www.tesla.com/findus/list/superchargers/United+States'

req_headers = {
    'casca'
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 axazs'
}

s = requests.Session()

r = s.get(url, headers=req_headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [3]:
address = soup.find_all('span', attrs={'class': 'street-address'})

In [4]:
locality = soup.find_all('span', attrs={'class': 'locality'})

In [5]:
ev_address = []
for i in address:
    if len(i)==0:
        ev_address.append('Coming-Soon')
    if len(i)==1:
        ev_address.append(str(i).split('>')[1].split('<')[0])

In [6]:
ev_locality = []
for j in locality:
    ev_locality.append(str(j).split('>')[1].split('<')[0])

In [7]:
tesla_ev = pd.DataFrame()
tesla_ev['Address'] = ev_address
tesla_ev['Locality'] = ev_locality

In [8]:
tesla_ev.head()

,Address,Locality
0,21282 Athens-Limestone Blvd,"Athens, AL"
1,1627 Opelika Road,"Auburn, AL 36830"
2,2221 Richard Arrington Junior Blvd,"Birmingham, AL 35203-1103"
3,Coming-Soon,"Dothan, AL"
4,219 Interstate Drive,"Greenville, AL 36037"


In [9]:
telsaEV_ca = tesla_ev[tesla_ev['Locality'].str.contains('CA')] #considering only California Supercharging Stations

In [10]:
i = telsaEV_ca[tesla_ev['Address'].str.contains('Coming-Soon')].index #drop upcoming Supercharing Stations

<ipython-input-10-438524c0d36e>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  i = telsaEV_ca[tesla_ev['Address'].str.contains('Coming-Soon')].index #drop upcoming Supercharing Stations


In [11]:
telsaEV_ca = telsaEV_ca.drop(i)

In [12]:
cityList = []
zipCodeList = []
for i in telsaEV_ca['Locality']:
    cityList.append(i.split(',')[0]) 
    
    if i.split()[-1].isnumeric() == True:
        zipCodeList.append(i.split()[-1])
    else:
        zipCodeList.append('0')

    
telsaEV_ca['City'] = cityList
telsaEV_ca['ZipCode'] = zipCodeList

In [13]:
telsaEV_ca = telsaEV_ca.drop(columns='Locality')

In [15]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
countyList = []

for i in telsaEV_ca['ZipCode']:
    date = search.by_zipcode(i)
    dataDict = date.to_dict()
    countyList.append(dataDict['county'])
    
telsaEV_ca['County'] = countyList

In [21]:
telsaEV_ca = telsaEV_ca.dropna()

In [24]:
telsaEV_ca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 46 to 976
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Address  179 non-null    object
 1   City     179 non-null    object
 2   ZipCode  179 non-null    object
 3   County   179 non-null    object
dtypes: object(4)
memory usage: 7.0+ KB


In [17]:
telsaEV_ca.to_csv("telsaEV_ca.csv")